**What I want to do: **Play the dogs versus cats competition on kaggle by fine-tuning a pretrained deep learning model, specifically the Vgg16 model.

Step 7 onwards. For Steps 1 to 6, please see the other notebook in the same directory.

## Step7: Rewrite the code taught during lesson 1 in the fastai course for the new dataset

### Admin stuff

In [4]:
%matplotlib inline

In [5]:
path = "data/data_kaggle_dogscats/sample/"
#path = "data/data_kaggle_dogscats/"

In [6]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

Jeremy has created a "utils.py" file with a whole host of convenience utility functions. I need to familiarise this in greater detail, especially the "plots" function. But lets simply use this for now.

In [7]:
import utils; reload(utils)
from utils import plots

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


### Use the pre-trained VGG model to classify our data

In [8]:
# define batch size 
batch_size = 64

Jeremy has tried to stick to the sklearn import instantiate fit predict paradigm by building a VGG class (I quickly looked online and I think the vgg class is a custom-built class). Let me run this step by step:

In [9]:
# import
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [10]:
# instantiate
vgg = Vgg16()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [11]:
type(vgg)

instance